# NLP를 활용한 레시피 추천 챗봇 구현
**개요**
- Collection 
- Preprocessing
- EDA
- Embeddings
- Modeling
- Streamlit

## 5. Modeling
**모델링 과정**
1. 사전 학습된 모델을 Sentence Transformer 를 통해 불러오기
2. 수집하고 기본적인 전처리를 거친 데이터를 위의 모델을 통해 임베딩 벡터로 변환한 파생변수 생성하기
3. input 으로 사용자가 재료를 포함한 문자열을 입력하면 문장을 벡터화하여 기존의 임베딩 벡터와 코사인 유사도를 구하는 함수
4. 코사인 유사도 상위 기준으로 n개를 output 으로 추천
5. OpenAI 의 API 를 활용하여 사용자의 문장 형태 input 의 의도를 파악하여 모델링 함수를 실행하도록 연동
6. streamlit 에 연동하여 배포


### 라이브러리 import

In [1]:
# 데이터 분석
import pandas as pd
import numpy as np

# 데이터 시각화
import matplotlib.pyplot as plt
import seaborn as sns

# 진행시간 표시
import swifter
from tqdm.notebook import tqdm
tqdm.pandas()

In [2]:
# 파이토치
import torch

# 문장 임베딩, transformer 유틸리티
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer, util
from sentence_transformers import SentenceTransformer, models

In [3]:
# 객체 복사
import copy

# JSON 형식 데이터 처리
import json

# 데이터 수집
import requests

In [4]:
# 데이터베이스 활용
import sqlite3 
import pickle

In [5]:
# OpenAI API 활용
import openai 
import os # 운영체제
import sys # 파이썬 변수, 함수 엑세스 
from dotenv import load_dotenv # 환경 변수 로드(API Key 보안)

os.environ["TOKENIZERS_PARALLELISM"] = "false"
load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

In [6]:
# 실행 os 확인
cur_os = sys.platform
cur_os

'win32'

### 데이터 불러오기

In [7]:
df = pd.read_pickle('data/compact_kosroberta_recipes.pkl')
df.shape

(4947, 13)

### 파생 변수
- feature1 = '재료'
- feature2 = '재료' + '요리'
- feature3 = '재료' + '요리' + '종류'
- feature4 = '재료' + '요리' + '종류' + '난이도'
- feature5 = '재료' + '요리' + '종류' + '난이도' + '요리방법'
- **feature = '재료' + '요리' + '설명' + '종류' + '난이도' + '요리방법'**

### 모델 불러오기
- model : Sentence Transformer
- pre trained model : `jhgan/ko-sroberta-multitask`

In [8]:
model_name = 'jhgan/ko-sroberta-multitask'

In [9]:
model = SentenceTransformer(model_name)
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: RobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [10]:
df['feature'][0]

"소시지 올리브오일 쌀 치킨스톡파우더 물 후춧가루 파마산치즈 파슬리 올리브 샬롯올리브 소시지 솥밥올리브를 넣어 이국적인 감칠맛이 매력적인 솥 밥. 별다른 양념없이도 술술 넘어가는 짭조름한 소시지가 바로 치트키랍니다~ 잘 익은 솥 밥에 파마산 치즈까지 듬뿍 뿌려 환상의 조화를 느껴보세요. 메인요리쉬움['쌀은 씻어 채에 받쳐 30분 정도 불려주세요.\\xa0', '소세지는 모양대로 슬라이스 해주세요. 샬롯은 잘게 썰어주세요. 파슬리는 다져주세요.', '냄비에 올리브오일을 두르고 다진 샬롯을 볶다가 쌀과 물, 치킨스톡을 넣고 센불에서 저어가며 3분 정도 끓여주세요. 뚜껑을 덮고 중약불에서 10분 정도 밥을 지어주세요.\\xa0', '그린올리브와 썰어둔 소세지를 올려 뚜껑을 덮고 3분정도 익힌 후 불을 끄로 10분간 뜸을 들여주세요.\\xa0', '파마산 치즈와 다진 파슬리를 뿌리고 골고루 섞어주세요. 기호에 따라 올리브오일, 후추를 뿌려 맛있게 즐겨주세요. \\r\\n']일반"

## 콘텐츠 추천 함수(코사인 유사도 기반)

In [33]:
def get_similar_recipes(query, model, df, exclude_ingredients=None, health=None, intent=None):
    data = {'요리': df['요리'], 'feature': df['ko-sroberta-multitask-feature']}
    query_encode = model.encode(query)
    query_encode = np.array(query_encode)  # NumPy 배열로 변환
    cos_scores = util.pytorch_cos_sim(query_encode, data['feature'])[0]
    top_results = torch.topk(cos_scores, k=100)

    top_indices = top_results[1].numpy()

    # exclude_ingredients 조건 적용
    if exclude_ingredients:
        if len(exclude_ingredients) > 1:
            exclude_mask = np.array([any(exclude in row for exclude in exclude_ingredients) for row in df['재료']])
        else:
            exclude_mask = np.array([exclude_ingredients[0] in row for row in df['재료']])
        exclude_idx = np.where(exclude_mask)[0]
        top_indices = np.setdiff1d(top_indices, exclude_idx)

    # health 조건 적용
    if health:
        include_mask = np.array([any(h in row for h in health) for row in df['질병']])
        include_idx = np.where(include_mask)[0]
        top_indices = np.intersect1d(top_indices, include_idx)

    filtered_scores = cos_scores[top_indices]
    top_filtered_results = torch.topk(torch.tensor(filtered_scores).clone().detach(), k=len(filtered_scores))

    if intent == '1':
        result_df = df.iloc[top_indices[top_filtered_results.indices], :][['요리', '종류', '재료', '설명', '난이도', '링크']]
        result_df = result_df.drop_duplicates(subset=['링크']).head(3)
    elif intent == '2':
        result_df = df.iloc[top_indices[top_filtered_results.indices], :][['요리', '종류', '재료', '설명', '난이도', '링크']]
        result_df = result_df.drop_duplicates(subset=['링크']).sample(3)
    else:
        result_df = df.iloc[top_indices[top_filtered_results.indices], :][['요리', '종류', '재료', '설명', '난이도', '링크']]
        result_df = result_df.drop_duplicates(subset=['링크']).head(5)
    
    return result_df

In [15]:
print(df.columns)

Index(['요리', '종류', '난이도', '재료', '요리방법', '설명', '재료수', '소요시간', '링크', '사진', '질병',
       'feature', 'ko-sroberta-multitask-feature'],
      dtype='object')


In [17]:
# 일반 추천
include = '여름 메뉴 추천해줘'
intent = '1'
exclude = []
health = []
result = get_similar_recipes(include, model, df, exclude_ingredients=exclude, health=health, intent=intent)
result

,요리,종류,재료,설명,난이도,링크
4074,파인애플 오이 샐러드,안주,라임즙 양파 소금 라임제스트 올리브오일 고수 라임 후춧가루 파인애플 오이 메이플시럽,Welcome! 우식 와인 Bar!\r\n 말만 들어도 시원해지는 샐러드 레시피!...,쉬움,https://wtable.co.kr/recipes/Mb7rr7JyP9CnKAiin...
2322,[장진아] 썸머 토스트,간단요리,홀그레인머스터드 올리브오일 참나물 매실청 그릭요거트 참외 먹물치아바타,여름에 가장 충만한 맛과 영양을 가진 제철 나물과 과일이 만났어요. 참나물과 미니참...,쉬움,https://wtable.co.kr/recipes/BzmvhcC28CxYobRga...
2111,용과 샐러드,간단요리,소금 화이트인식초 올리브오일 용과 파파야,비주얼만으로도 산뜻한 여름을 느낄 수 있는 용과 샐러드! 부드러운 식감의 용과에 향...,쉬움,https://wtable.co.kr/recipes/yvM5LWPJZmYYyoPZE...


In [18]:
# 재추천
include = '여름 메뉴 추천해줘'
intent = '2'
exclude = []
health = []
result = get_similar_recipes(include, model, df, exclude_ingredients=exclude, health=health, intent=intent)
result

,요리,종류,재료,설명,난이도,링크
4537,미역 콩국수,다이어트,미역국수 소금 얼음 두부 볶은땅콩 두유 오이 통깨 방울토마토,"한 번 맛보면 빠져나올 수 없는 맛, 콩국수! 시원하고 고소한 맛 덕분에 여름철 ...",쉬움,https://wtable.co.kr/recipes/Y6BSSm38nwV7q8Bph...
4538,미역 비빔국수,다이어트,비빔소스 매실청 당근 해조미미역국수 오이 참기름 통깨,언제 먹어도 맛있는 비빔국수! 다만 밀가루 면을 사용해서 자주 먹기에는 조금 부담...,쉬움,https://wtable.co.kr/recipes/yBejj8eRpyjMiuprF...
3371,오이무침,한식,소금 양파 설탕 다진마늘 고추장 올리고당 간장 식초 오이 고춧가루 참기름 통깨,시원하고 아삭한 여름 대표 채소! 수분이 풍부하고 부종제거에 효과적인 건강채소 오이...,쉬움,https://wtable.co.kr/recipes/jdQ71z77Ms4GkCzXm...


In [19]:
# 단일 제외
include = '여름 메뉴 추천해줘'
intent = '1'
exclude = ['오이']
health = []
result = get_similar_recipes(include, model, df, exclude_ingredients=exclude, health=health, intent=intent)
result

,요리,종류,재료,설명,난이도,링크
2322,[장진아] 썸머 토스트,간단요리,홀그레인머스터드 올리브오일 참나물 매실청 그릭요거트 참외 먹물치아바타,여름에 가장 충만한 맛과 영양을 가진 제철 나물과 과일이 만났어요. 참나물과 미니참...,쉬움,https://wtable.co.kr/recipes/BzmvhcC28CxYobRga...
2111,용과 샐러드,간단요리,소금 화이트인식초 올리브오일 용과 파파야,비주얼만으로도 산뜻한 여름을 느낄 수 있는 용과 샐러드! 부드러운 식감의 용과에 향...,쉬움,https://wtable.co.kr/recipes/yvM5LWPJZmYYyoPZE...
1805,토마토빙수,간식,설탕 완숙토마토 다진피스타치오 스노우요거트얼음 바질잎 플레인요거트 연유 토마토퓌레 우유,여름아 안녕! 여름이 점점 다가오고 있어요. 여름에 아이들이 가장 먼저 찾는 음식...,쉬움,https://wtable.co.kr/recipes/ZYJAE5LVy1wCv36Zi...


In [20]:
# 다중 제외
include = '여름 메뉴 추천해줘'
intent = '1'
exclude = ['오이', '토마토']
health = []
result = get_similar_recipes(include, model, df, exclude_ingredients=exclude, health=health, intent=intent)
result

,요리,종류,재료,설명,난이도,링크
2322,[장진아] 썸머 토스트,간단요리,홀그레인머스터드 올리브오일 참나물 매실청 그릭요거트 참외 먹물치아바타,여름에 가장 충만한 맛과 영양을 가진 제철 나물과 과일이 만났어요. 참나물과 미니참...,쉬움,https://wtable.co.kr/recipes/BzmvhcC28CxYobRga...
2111,용과 샐러드,간단요리,소금 화이트인식초 올리브오일 용과 파파야,비주얼만으로도 산뜻한 여름을 느낄 수 있는 용과 샐러드! 부드러운 식감의 용과에 향...,쉬움,https://wtable.co.kr/recipes/yvM5LWPJZmYYyoPZE...
15,상그리아 셔벗,메인요리,설탕 베리류 민트잎 물 상그리아,상그리아를 얼려서 만드는 시원한 셔벗.\r\n달콤한 과일향이 매력적인 여름 디저트에...,쉬움,https://wtable.co.kr/recipes/qaXLiNE5HiJ3ALgQ3...


In [34]:
# 건강 상태
include = '샐러드 레시피 알려줘'
intent = 1
exclude = []
health = ['비만']
result = get_similar_recipes(include, model, df, exclude_ingredients=exclude, health=health, intent=intent)
result

C:\Users\CHAEKANG\AppData\Local\Temp\ipykernel_20720\2061459323.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  top_filtered_results = torch.topk(torch.tensor(filtered_scores).clone().detach(), k=len(filtered_scores))


,요리,종류,재료,설명,난이도,링크
2286,달걀토마토 샐러드,간단요리,홀그레인머스터드 소금 다진양파 블랙올리브 올리브유 달걀 대추토마토 새싹채소 꿀 호두 식초,영양 가득한 재료들만 모아 특별한 샐러드를 만들었어요.\n비타민과 무기질이 가득한 ...,쉬움,https://wtable.co.kr/recipes/dYxnz6f3zPKb6Jcjq...
2130,꼬시래기 샐러드,간단요리,샐러드채소 현미식초 치킨스테이크 된장 매실액 꼬시래기,꼬시래기는 지방과 탄수화물 함량이 낮고 칼슘과 식이섬유가 많아 다이어트에 효과적인 ...,쉬움,https://wtable.co.kr/recipes/5cEMJ2c75qBM6x7iQ...
2151,닭가슴살 샐러드,간단요리,소금 다진양파 달걀 다진마늘 올리브오일 후춧가루 올리고당 간장 식초 양상추 당근 닭...,연초부터 다이어트 시작한 분들 많으시죠? 다이어터 분들이 식사로 가장 많이 찾게 되...,쉬움,https://wtable.co.kr/recipes/dGN5zEGYrh8kayxv5...
2216,배추땅콩칠리샐러드,간단요리,배추 칠리소스 후춧가루 사과식초 땅콩 부추,"배추는 씹으면 씹을수록 고소한 맛이 진하게 나서 생으로 먹기 좋고, 섬유질이 풍부한...",쉬움,https://wtable.co.kr/recipes/7eNiM43eZ8v5PDiQp...
2472,부라타샐러드,간단요리,소금 후추 올리브오일 바질잎 방울토마토 바질페스토 부라타치즈,몽글몽글 마치 순두부같이 부드러운 부라타 치즈로 만든 샐러드 입니다! 신선하고 건...,쉬움,https://wtable.co.kr/recipes/Vknh411PRVQVuxm6a...


## ChatGPT 연동하기
### 세 가지 Role
- user : 마치 채팅하는 것처럼 ChatGPT에 직접 무언가를 물어보고 싶을 때 사용하는 role (ex. "Please explain what AI is")
- system : 유저에게 메시지를 받기 전에 모델을 초기화하거나 구성하려는 경우 사용하는 role (ex. "You are a helpful kindergarten teacher talking to children")
- assistant : 이전에 ChatGPT가 유저에게 보낸 메시지가 무엇인지 알려주는 role. 유저와 어시스턴트 사이의 대화를 저장하고 어시스턴트에게 이전 대화를 전달하여 응답값을 조정할 수 있음. (ex. 이전까지의 대화를 기억하게 함으로써 명사 -> 대명사로 이어지는 맥락을 이해할 수 있도록 해줌)
### Fine tuning
Role 을 지정하는 것 이외에 Fine-tuning 을 활용하는 것이 실제 서비스에서는 더 이상적인 형태
사용자의 데이터를 가지고 특정 태스크에 알맞게 커스텀하는 것
- `{"prompt" : "I really like this recipe!", "completion" : "positive"}`
- `{"prompt" : "I'd like to leave out the cucumber.", "completion" : "negative"}`